In [155]:
!pip3 install numpy



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [156]:
from moralis import evm_api

params = {
  "chain": "eth",
  "address": "0x1f9090aaE28b8a3dCeaDf281B0F12828e676c326"
}

wallet_history = evm_api.token.get_wallet_token_balances(
  api_key=MORALIS_API_KEY,
  params=params,
)

# Initialize the OpenAI embedding model
model_name = 'text-embedding-ada-002'


In [212]:

from langchain.embeddings import OpenAIEmbeddings
from openai import OpenAI
from pinecone import Pinecone
from pinecone import ServerlessSpec
import numpy as np

# for vector store
embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

client = OpenAI(api_key=OPENAI_API_KEY)

def json_to_embedding(json_data):
    text_data = []

    # Example extraction from JSON
    for item in json_data:
        transaction_details = f"Token address: {item['token_address']}, Token Balance: {item['balance']}, Symbol: {item['symbol']}"
        text_data.append(transaction_details)

    embedding = client.embeddings.create(input=text_data, model=model_name).data[0].embedding
    
    return text_data, embedding

pc = Pinecone(api_key=PINECONE_API_KEY)

# # Create a new index
index_name = 'wallet-history-index'

text_data, embeddings = json_to_embedding(wallet_history)

# Store embeddings
# pc.create_index(index_name, dimension=len(embeddings),spec=ServerlessSpec(
#             cloud='aws', 
#             region='us-east-1'
#         ) )
index = pc.Index(index_name)

index.upsert([
        {"id": "vec1", "values": embeddings, "metadata" : {"text": text_data}}
    ],namespace="Token_Balance")

[-0.007455748971551657, -0.0343814343214035, -0.022302862256765366, -0.03069862723350525, -0.03208933770656586, 0.002618847880512476, -0.038347531110048294, 0.001946027739904821, -0.007262594532221556, -0.003892055479809642, 0.02329438552260399, 0.005652977153658867, -0.008782073855400085, -0.019096503034234047, 0.00590085843577981, 0.020152412354946136, 0.008949474431574345, -0.00463569862768054, 0.010346622206270695, 0.003262694925069809, -0.010771561414003372, 0.015310683287680149, -0.01959870383143425, 0.010790876112878323, -0.012825433164834976, 0.011627878062427044, 0.01680440828204155, -0.004165690392255783, 0.0009625513339415193, -0.046769049018621445, -0.00018691684817895293, 0.0003577375318855047, -0.030621366575360298, -0.025431958958506584, -0.018748825415968895, 0.013456403277814388, -0.007526571862399578, -0.027711177244782448, 0.029745733365416527, -0.011151431128382683, 0.008112472482025623, 0.019392672926187515, -0.0028667289298027754, 0.0025077841710299253, -0.0101534

{'upserted_count': 1}

In [213]:
query = "What is the token balance of address 0xdAC17F958D2ee523a2206206994597C13D831ec7?"

query_embedding = client.embeddings.create(input=[query], model=model_name).data[0].embedding

results = index.query(namespace="Token_Balance",vector=[query_embedding], top_k=1,include_values=True, include_metadata=True)

relevant_texts = [match['metadata']['text'] for match in results['matches']]

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Using the following data: {relevant_texts}. Answer the question: {query}"}
  ]
)

print(response.choices[0].message.content)

{'namespace': '', 'vectors': {}}
The token balance of address 0xdAC17F958D2ee523a2206206994597C13D831ec7 is 211,455,461.
